# Sentiment analysis

## Reading the File

In [2]:
import pandas as pd
df = pd.read_json( 'Arts_Crafts_and_Sewing_5.json',lines=True)
df.sample(10)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
1847,5,True,"03 18, 2015",A3OGBKXV009WME,B00006IE0H,{'Package Quantity:': ' 1'},J. Lindsey,"Nice paper, like the colors!",Five Stars,1426636800,NaN,NaN
158487,5,True,"04 28, 2014",A3KOVJZ11FKJHQ,B001R4T4VE,NaN,Connie L. Dorathy,These are the best bobbin boxes around. Becau...,I love these boxes,1398643200,NaN,NaN
38916,5,True,"01 30, 2017",A4Z1IISIRIG2X,B000UU6SR4,NaN,waimeawahine,Sharp. Great scissors.,Sewer's Dream,1485734400,NaN,NaN
352831,5,True,"02 21, 2015",A24OQGHC97G2ED,B00NTMZS2G,NaN,Lorraine R. Vaught,These little dolls are the exact replica of Ju...,So cute,1424476800,NaN,NaN
69102,5,True,"07 18, 2016",ARDWRMCC47XRF,B00139SZU4,{'Size:': ' 2 oz'},Amazon Customer,I have never used masking fluid before so have...,but seems to work just fine. Thank you for mak...,1468800000,2,NaN
143403,5,True,"11 15, 2014",A38IQNFUVTA4OS,B001DEJJV8,NaN,JohnQ,Have used these types of needles for years wit...,Great for sewing with larger threads that migh...,1416009600,NaN,NaN
452916,4,True,"07 12, 2016",A2AJ3BEG1YNM6A,B0081A373S,NaN,Kurrjur,Aside from a few minor blemishes these canvase...,Four Stars,1468281600,NaN,NaN
217898,5,True,"02 14, 2016",A1IQPX7LRNQQOT,B004V09CNQ,NaN,MonkeeBumz,great size,Five Stars,1455408000,NaN,NaN
380818,5,True,"09 24, 2017",AFBQBSW19KOXK,B0109326CM,NaN,Anthony Reeder,Wonderful little knives,Five Stars,1506211200,NaN,NaN
103909,5,True,"06 1, 2017",A1TYRMVV8BBUX5,B000YZ7R2K,{'Color:': ' 18mm'},Mitchell Joe,Item as described - fast shipping - thanks!,Five Stars,1496275200,NaN,NaN


## Importing Bing liu lexicon

In [4]:
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\gragh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\opinion_lexicon.zip.


In [5]:
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',
opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


## Creating dictionary

In [6]:
nltk.download('punkt')
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gragh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


## Adding the positive words to the dictionary

In [8]:
for word in opinion_lexicon.positive():
    word_dict[word] = pos_score

## Adding the negative words to the dictionary

In [9]:
for word in opinion_lexicon.negative():
    word_dict[word] = neg_score

## Function to do the math

In [10]:
def bing_liu_score(text, word_dict):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
    return sentiment_score

In [12]:
df['text'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['text'].apply(lambda x: bing_liu_score(x, word_dict))

In [13]:
df[['overall',"text", 'Bing_Liu_Score']].head(10)

,overall,text,Bing_Liu_Score
0,4,Contains some interesting stitches.,1
1,5,I'm a fairly experienced knitter of the one-co...,22
2,4,Great book but the index is terrible. Had to w...,0
3,5,I purchased the Kindle edition which is incred...,4
4,5,Very well laid out and very easy to read.\n\nT...,5
5,5,"Beginning her career as a freelance knitter, M...",15
6,5,This is a terrific stitch handbook (and I have...,9
7,4,The book needs to be coil bound. The content i...,1
8,5,I really am enjoying this book! I like the siz...,12
9,5,Just received this book and looked over it cov...,6


In [14]:
df.groupby('overall').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
overall,
1,-0.255049
2,0.566098
3,1.158796
4,2.028146
5,2.130005
